# Qooked Tahoe Quatum Challenge 2025

## Resource Allocation Problem
Resource Allocation Problem:

### Rewriting MIS into QUBO
The Quadratic Unconstrained Binary Optimization(QUBO) framework can be used to express our MIS problem as we want to minimize $x^TQx$ where $x$ is our vector of binary variables, $Q$ is our matrix encoding high level of wildfire threat. We first encapsulate our geospatial map of our desired location using weighted fire-threat analysis in to our matrix:

QUBO Formalism using PyQUBO:

In [8]:
import numpy as np
from pyqubo import Binary
from neal import SimulatedAnnealingSampler

def matrix_to_qubo(matrix, alpha=5, beta=4, allow_adjacent_penalty=True):
    n = matrix.shape[0]
    variables = {}

    for i in range(n):
        for j in range(n):
            variables[(i, j)] = Binary(f"x{i}{j}")

    H = -sum(matrix[i][j] * variables[(i, j)] for i in range(n) for j in range(n))

    total_selected = sum(variables[(i, j)] for i in range(n) for j in range(n))
    H += alpha * total_selected

    if allow_adjacent_penalty:
        for i in range(n):
            for j in range(n):
                if i < n - 1:
                    H += beta * variables[(i, j)] * variables[(i + 1, j)]
                if j < n - 1:
                    H += beta * variables[(i, j)] * variables[(i, j + 1)]

    model = H.compile()
    qubo, offset = model.to_qubo()
    return qubo, offset, model

Solving QUBO with Quantum Annealing:

In [9]:
def solve_qubo_with_neal(matrix, alpha=5, beta=4, num_reads=100):
    qubo, offset, model = matrix_to_qubo(matrix, alpha=alpha, beta=beta)
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample_qubo(qubo, num_reads=num_reads)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = decoded_samples[0]

    n = matrix.shape[0]
    solution_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            var_name = f"x{i}{j}"
            solution_matrix[i, j] = best_sample.sample.get(var_name, 0)

    return solution_matrix, best_sample.energy + offset


matrix =  np.random.randint(0, 9, (10, 10))

solution, energy = solve_qubo_with_neal(matrix, alpha=7, beta=4)
print("Wildfire Intensity Matrix:\n", matrix)
print("Optimal Response Deployment:\n", solution)
print("Energy (QUBO Objective):", energy)
print("Units used:", int(solution.sum()))

Wildfire Intensity Matrix:
 [[7 1 8 8 7 2 7 3 5 3]
 [6 8 0 8 3 4 0 8 1 2]
 [1 0 5 6 3 2 3 2 5 4]
 [1 5 4 7 8 5 1 7 6 1]
 [5 6 6 1 8 0 1 3 8 8]
 [6 0 6 7 7 4 6 6 7 5]
 [6 5 1 5 7 0 7 2 5 0]
 [2 7 8 8 3 2 6 6 2 0]
 [5 1 1 5 6 7 8 8 8 3]
 [5 3 1 5 6 8 6 4 3 5]]
Optimal Response Deployment:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
Energy (QUBO Objective): -10.0
Units used: 11
